## 1️⃣ Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ⚠️ SCHIMBĂ CALEA CĂTRE FOLDERUL CU DATASET-UL PE GOOGLE DRIVE
DATASET_FOLDER = '/content/drive/My Drive/deepfake_dataset'

# Verifică dacă există ZIP-uri și extrage-le
import os
import zipfile

print(f"📁 Conținut folder: {os.listdir(DATASET_FOLDER)}")

# Caută fișiere ZIP
zip_files = [f for f in os.listdir(DATASET_FOLDER) if f.endswith('.zip')]

if zip_files:
    print(f"\n📦 Găsite {len(zip_files)} fișiere ZIP:")
    for zip_file in zip_files:
        zip_path = os.path.join(DATASET_FOLDER, zip_file)
        print(f"   Extrag: {zip_file}...")
        
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(DATASET_FOLDER)
        
        print(f"   ✅ Extras: {zip_file}")
    
    print(f"\n📁 Conținut după extragere: {os.listdir(DATASET_FOLDER)}")

# Verifică structura finală
DATASET_PATH = os.path.join(DATASET_FOLDER, 'train')

# Dacă train/ nu există, verifică dacă real/ și fake/ sunt direct în folder
if not os.path.exists(DATASET_PATH):
    print(f"\n⚠️ Folder 'train/' nu găsit, verific dacă real/fake sunt direct în {DATASET_FOLDER}")
    if os.path.exists(os.path.join(DATASET_FOLDER, 'real')) and os.path.exists(os.path.join(DATASET_FOLDER, 'fake')):
        DATASET_PATH = DATASET_FOLDER
        print(f"✅ Găsite real/ și fake/ direct în {DATASET_FOLDER}")

if os.path.exists(DATASET_PATH):
    print(f"\n✅ Dataset final: {DATASET_PATH}")
    real_path = os.path.join(DATASET_PATH, 'real')
    fake_path = os.path.join(DATASET_PATH, 'fake')
    
    if os.path.exists(real_path) and os.path.exists(fake_path):
        real_count = len(os.listdir(real_path))
        fake_count = len(os.listdir(fake_path))
        print(f"   Real: {real_count:,} imagini")
        print(f"   Fake: {fake_count:,} imagini")
        print(f"   Total: {real_count + fake_count:,} imagini")
    else:
        print(f"❌ Folderele real/ sau fake/ nu există în {DATASET_PATH}")
        print(f"   Structură găsită: {os.listdir(DATASET_PATH)}")
else:
    print(f"❌ Dataset NU găsit. Structură folder:")
    print(f"   {os.listdir(DATASET_FOLDER)}")

## 2️⃣ Verifică GPU

In [ ]:
!nvidia-smi -L

import tensorflow as tf
print(f"\n✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")

## 3️⃣ Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("✅ Toate librăriile importate")

## 4️⃣ Configurare Training

In [ ]:
CONFIG = {
    'input_shape': (256, 256, 3),
    'batch_size': 32,  # GPU T4 poate 32, vs 16 pe CPU
    'dropout_rate': 0.5,
    'train_split': 0.8,
    
    # Training phases
    'initial_epochs': 10,
    'fine_tune_epochs': 10,
    'learning_rate': 0.001,
    'fine_tune_layers': 20,
    
    # Data augmentation
    'augmentation': {
        'rotation_range': 20,
        'width_shift_range': 0.2,
        'height_shift_range': 0.2,
        'horizontal_flip': True,
        'zoom_range': 0.2,
        'brightness_range': [0.8, 1.2],
        'shear_range': 0.15,
        'fill_mode': 'nearest'
    }
}

print("✅ Configurare setată")
print(f"   Input shape: {CONFIG['input_shape']}")
print(f"   Batch size: {CONFIG['batch_size']}")
print(f"   Total epochs: {CONFIG['initial_epochs'] + CONFIG['fine_tune_epochs']}")

## 5️⃣ Data Generators

In [ ]:
# Training data (with augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,  # ⚠️ IMPORTANT: Normalize 0-255 to 0-1
    validation_split=1-CONFIG['train_split'],
    **CONFIG['augmentation']
)

# Validation data (no augmentation)
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=1-CONFIG['train_split']
)

# Training generator
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=CONFIG['input_shape'][:2],
    batch_size=CONFIG['batch_size'],
    class_mode='binary',
    subset='training',
    shuffle=True,
    seed=42
)

# Validation generator
val_generator = val_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=CONFIG['input_shape'][:2],
    batch_size=CONFIG['batch_size'],
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"\n✅ Training samples: {train_generator.samples:,}")
print(f"✅ Validation samples: {val_generator.samples:,}")
print(f"✅ Steps per epoch: {len(train_generator):,}")
print(f"\nClass indices: {train_generator.class_indices}")

## 6️⃣ Create Xception Model (Transfer Learning)

In [ ]:
def create_model():
    # Load Xception pre-trained on ImageNet
    base_model = Xception(
        include_top=False,  # Remove classification head
        weights='imagenet',  # Use pre-trained weights
        input_shape=CONFIG['input_shape']
    )
    
    # Freeze base model (Phase 1)
    base_model.trainable = False
    
    # Build complete model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(CONFIG['dropout_rate']),
        layers.Dense(1, activation='sigmoid')  # Binary: 0=Real, 1=Fake
    ], name='xception_deepfake_detector')
    
    return model, base_model

# Create model
model, base_model = create_model()

# Compile for Phase 1
model.compile(
    optimizer=keras.optimizers.Adam(CONFIG['learning_rate']),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print("\n" + "="*60)
print("MODEL SUMMARY")
print("="*60)
model.summary()

total_params = model.count_params()
trainable_params = sum([tf.size(var).numpy() for var in model.trainable_variables])
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")

## 7️⃣ PHASE 1: Train Classification Head (Frozen Base)

In [ ]:
print("\n" + "="*60)
print("PHASE 1: TRAINING CLASSIFICATION HEAD (BASE FROZEN)")
print("="*60)
print(f"Epochs: {CONFIG['initial_epochs']}")
print(f"Learning rate: {CONFIG['learning_rate']}")
print("="*60 + "\n")

# Callbacks
callbacks_phase1 = [
    keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/My Drive/deepfake_phase1_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

# Train Phase 1
history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=CONFIG['initial_epochs'],
    callbacks=callbacks_phase1,
    verbose=1
)

print("\n✅ Phase 1 completed!")

## 8️⃣ PHASE 2: Fine-Tune Top Layers (Unfreeze)

In [ ]:
print("\n" + "="*60)
print("PHASE 2: FINE-TUNING (UNFREEZE TOP LAYERS)")
print("="*60)

# Unfreeze top layers
base_model.trainable = True
for layer in base_model.layers[:-CONFIG['fine_tune_layers']]:
    layer.trainable = False

print(f"✅ Unfroze last {CONFIG['fine_tune_layers']} layers")
trainable_params = sum([tf.size(var).numpy() for var in model.trainable_variables])
print(f"   Trainable params: {trainable_params:,}")

# Recompile with lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(CONFIG['learning_rate'] / 10),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc')
    ]
)

print(f"\nEpochs: {CONFIG['fine_tune_epochs']}")
print(f"Learning rate: {CONFIG['learning_rate'] / 10}")
print("="*60 + "\n")

# Callbacks Phase 2
callbacks_phase2 = [
    keras.callbacks.ModelCheckpoint(
        filepath='/content/drive/My Drive/deepfake_xception_final.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-8,
        verbose=1
    )
]

# Train Phase 2
history_phase2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=CONFIG['fine_tune_epochs'],
    callbacks=callbacks_phase2,
    verbose=1
)

print("\n✅ Phase 2 completed!")

## 9️⃣ Evaluate Final Model

In [ ]:
print("\n" + "="*60)
print("FINAL EVALUATION")
print("="*60)

results = model.evaluate(val_generator, verbose=1)

print(f"\n📊 Final Results:")
print(f"   Loss: {results[0]:.4f}")
print(f"   Accuracy: {results[1]*100:.2f}%")
print(f"   Precision: {results[2]*100:.2f}%")
print(f"   Recall: {results[3]*100:.2f}%")
print(f"   AUC: {results[4]:.4f}")
print("="*60)

## 🔟 Plot Training History

In [ ]:
# Combine histories
history = {
    'phase1': history_phase1.history,
    'phase2': history_phase2.history
}

# Plot
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history_phase1.history['accuracy'], label='Phase 1 Train')
axes[0, 0].plot(history_phase1.history['val_accuracy'], label='Phase 1 Val')
axes[0, 0].plot(range(len(history_phase1.history['accuracy']), 
                      len(history_phase1.history['accuracy']) + len(history_phase2.history['accuracy'])),
                history_phase2.history['accuracy'], label='Phase 2 Train')
axes[0, 0].plot(range(len(history_phase1.history['accuracy']), 
                      len(history_phase1.history['accuracy']) + len(history_phase2.history['accuracy'])),
                history_phase2.history['val_accuracy'], label='Phase 2 Val')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Training & Validation Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Loss
axes[0, 1].plot(history_phase1.history['loss'], label='Phase 1 Train')
axes[0, 1].plot(history_phase1.history['val_loss'], label='Phase 1 Val')
axes[0, 1].plot(range(len(history_phase1.history['loss']), 
                      len(history_phase1.history['loss']) + len(history_phase2.history['loss'])),
                history_phase2.history['loss'], label='Phase 2 Train')
axes[0, 1].plot(range(len(history_phase1.history['loss']), 
                      len(history_phase1.history['loss']) + len(history_phase2.history['loss'])),
                history_phase2.history['val_loss'], label='Phase 2 Val')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].set_title('Training & Validation Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Precision
axes[1, 0].plot(history_phase1.history['precision'], label='Phase 1 Train')
axes[1, 0].plot(history_phase2.history['precision'], label='Phase 2 Train')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True)

# Recall
axes[1, 1].plot(history_phase1.history['recall'], label='Phase 1 Train')
axes[1, 1].plot(history_phase2.history['recall'], label='Phase 2 Train')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].set_title('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('/content/drive/My Drive/training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Grafice salvate pe Google Drive")

## 1️⃣1️⃣ Save Metadata

In [ ]:
metadata = {
    'model_type': 'Transfer Learning - Xception',
    'dataset': '140k Real and Fake Faces',
    'training_date': datetime.now().isoformat(),
    'training_platform': 'Google Colab GPU T4',
    'architecture': 'xception',
    'input_shape': CONFIG['input_shape'],
    'batch_size': CONFIG['batch_size'],
    'initial_epochs': CONFIG['initial_epochs'],
    'fine_tune_epochs': CONFIG['fine_tune_epochs'],
    'total_epochs': CONFIG['initial_epochs'] + CONFIG['fine_tune_epochs'],
    'training_samples': train_generator.samples,
    'validation_samples': val_generator.samples,
    'class_names': {v: k for k, v in train_generator.class_indices.items()},
    'final_metrics': {
        'loss': float(results[0]),
        'accuracy': float(results[1]),
        'precision': float(results[2]),
        'recall': float(results[3]),
        'auc': float(results[4])
    }
}

# Save to Google Drive
with open('/content/drive/My Drive/deepfake_xception_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Metadata salvat pe Google Drive")
print(f"\n📁 Fișiere salvate:")
print(f"   1. /content/drive/My Drive/deepfake_xception_final.keras")
print(f"   2. /content/drive/My Drive/deepfake_xception_metadata.json")
print(f"   3. /content/drive/My Drive/training_history.png")
print(f"\n🎉 TRAINING COMPLET! Descarcă modelul de pe Google Drive.")

## 1️⃣2️⃣ Download Model (Opțional)

In [ ]:
# Download direct în browser (dacă vrei)
from google.colab import files

print("📥 Descarcă fișierele...")
files.download('/content/drive/My Drive/deepfake_xception_final.keras')
files.download('/content/drive/My Drive/deepfake_xception_metadata.json')
files.download('/content/drive/My Drive/training_history.png')
print("✅ Download complet!")